In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, Dataset 
import torch
import torch.nn as nn
from torch.autograd import Variable 

c:\Users\Timii\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
pd_xela_allfiles = pd.read_csv('data.csv', index_col=0)
pd_sliplabel_allfiles = pd.read_csv('labels.csv', index_col=0)

data = pd_xela_allfiles.to_numpy()
labels = pd_sliplabel_allfiles.to_numpy()

In [ ]:
sc = StandardScaler()
sc.fit(data)
data = sc.transform(data)

In [ ]:
data_prime = []
for t in range(data.shape[0]-9):
    data_prime.append(data[t:t+10])
data_prime = np.array(data_prime)
label_prime = labels[9:]
# data_prime_v = data_prime[:]
# label_prime = label_prime[:]

In [ ]:
X_train_tensors = Variable(torch.Tensor(data_prime))
y_train_tensors = Variable(torch.Tensor(label_prime))

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 16) #fully connected 1
        self.fc = nn.Linear(16, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    
    def forward(self,x):
        h_0 = Variable(torch.zeros(1, self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(1, self.hidden_size)) #internal state
        hn, cn = h_0, c_0
        # Propagate input through LSTM
        for t in range(self.seq_length):
            output, (hn, cn) = self.lstm(x[:,t,:], (hn, cn)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        out = self.sigmoid(out)
        return out

In [ ]:
num_epochs = 3 #1000 epochs
learning_rate = 0.001 #0.001 lr

input_size = 48 #number of features
hidden_size = 16 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes

lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]) #our lstm class 
lstm1.seq_length = 10
criterion = torch.nn.BCELoss()   # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    outputs = lstm1.forward(X_train_tensors) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
 
  # obtain the loss function
    loss = criterion(outputs, y_train_tensors)
    
    loss.backward() #calculates the loss of the loss function
    
    optimizer.step() #improve from loss, i.e backprop
    if epoch % 1 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))